In [8]:
import tkinter as tk
from tkinter import messagebox

def add_task():
    task = task_entry.get()
    if task != "":
        task_listbox.insert(tk.END, task)
        task_entry.delete(0, tk.END)
    else:
        messagebox.showwarning("Warning", "You must enter a task!")

def delete_task():
    try:
        selected_task_index = task_listbox.curselection()[0]
        task_listbox.delete(selected_task_index)
    except IndexError:
        messagebox.showwarning("Warning", "You must select a task to delete!")

def save_tasks():
    tasks = task_listbox.get(0, tk.END)
    with open("tasks.txt", "w") as f:
        for task in tasks:
            f.write(task + "\n")
    messagebox.showinfo("Success", "Tasks saved successfully!")

def load_tasks():
    try:
        with open("tasks.txt", "r") as f:
            tasks = f.readlines()
            for task in tasks:
                task_listbox.insert(tk.END, task.strip())
    except FileNotFoundError:
        pass

# GUI Setup
root = tk.Tk()
root.title("To-Do List")

frame = tk.Frame(root)
frame.pack(pady=10)

task_listbox = tk.Listbox(frame, width=50, height=10)
task_listbox.pack(side=tk.LEFT, fill=tk.BOTH)

scrollbar = tk.Scrollbar(frame)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

task_listbox.config(yscrollcommand=scrollbar.set)
scrollbar.config(command=task_listbox.yview)

task_entry = tk.Entry(root, width=50)
task_entry.pack(pady=10)

button_frame = tk.Frame(root)
button_frame.pack(pady=10)

add_button = tk.Button(button_frame, text="Add Task", command=add_task)
delete_button = tk.Button(button_frame, text="Delete Task", command=delete_task)
save_button = tk.Button(button_frame, text="Save Tasks", command=save_tasks)

add_button.pack(side=tk.LEFT, padx=5)
delete_button.pack(side=tk.LEFT, padx=5)
save_button.pack(side=tk.LEFT, padx=5)

load_tasks()
root.mainloop()


In [9]:
# the devolpment one 
import tkinter as tk
from tkinter import messagebox, simpledialog
import sqlite3

# Database setup
def init_db():
    conn = sqlite3.connect("tasks.db")
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS tasks (
            id INTEGER PRIMARY KEY,
            task TEXT NOT NULL,
            completed BOOLEAN NOT NULL DEFAULT 0,
            priority TEXT NOT NULL DEFAULT 'Medium',
            due_date TEXT
        )
    """)
    conn.commit()
    conn.close()

def add_task():
    task = task_entry.get()
    if task != "":
        priority = priority_var.get()
        due_date = simpledialog.askstring("Due Date", "Enter due date (optional):")
        conn = sqlite3.connect("tasks.db")
        cursor = conn.cursor()
        cursor.execute("INSERT INTO tasks (task, priority, due_date) VALUES (?, ?, ?)", (task, priority, due_date))
        conn.commit()
        conn.close()
        load_tasks()
        task_entry.delete(0, tk.END)
    else:
        messagebox.showwarning("Warning", "You must enter a task!")

def delete_task():
    try:
        selected_task = task_listbox.get(task_listbox.curselection())
        task_id = selected_task.split(".")[0]
        conn = sqlite3.connect("tasks.db")
        cursor = conn.cursor()
        cursor.execute("DELETE FROM tasks WHERE id = ?", (task_id,))
        conn.commit()
        conn.close()
        load_tasks()
    except IndexError:
        messagebox.showwarning("Warning", "You must select a task to delete!")

def toggle_task_completion():
    try:
        selected_task = task_listbox.get(task_listbox.curselection())
        task_id = selected_task.split(".")[0]
        conn = sqlite3.connect("tasks.db")
        cursor = conn.cursor()
        cursor.execute("UPDATE tasks SET completed = NOT completed WHERE id = ?", (task_id,))
        conn.commit()
        conn.close()
        load_tasks()
    except IndexError:
        messagebox.showwarning("Warning", "You must select a task to mark as complete!")

def edit_task():
    try:
        selected_task = task_listbox.get(task_listbox.curselection())
        task_id = selected_task.split(".")[0]
        new_task = simpledialog.askstring("Edit Task", "Enter new task text:")
        if new_task:
            conn = sqlite3.connect("tasks.db")
            cursor = conn.cursor()
            cursor.execute("UPDATE tasks SET task = ? WHERE id = ?", (new_task, task_id))
            conn.commit()
            conn.close()
            load_tasks()
    except IndexError:
        messagebox.showwarning("Warning", "You must select a task to edit!")

def load_tasks():
    task_listbox.delete(0, tk.END)
    conn = sqlite3.connect("tasks.db")
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM tasks")
    tasks = cursor.fetchall()
    conn.close()
    for task in tasks:
        display_text = f"{task[0]}. {task[1]} [Priority: {task[3]}] [Due: {task[4] if task[4] else 'N/A'}]"
        if task[2]:
            display_text += " ✅"
        task_listbox.insert(tk.END, display_text)

# GUI Setup
root = tk.Tk()
root.title("To-Do List")
root.configure(bg="#121212")

frame = tk.Frame(root, bg="#121212")
frame.pack(pady=10)

task_listbox = tk.Listbox(frame, width=60, height=10, bg="#1E1E1E", fg="white")
task_listbox.pack(side=tk.LEFT, fill=tk.BOTH)

scrollbar = tk.Scrollbar(frame)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

task_listbox.config(yscrollcommand=scrollbar.set)
scrollbar.config(command=task_listbox.yview)

task_entry = tk.Entry(root, width=50, bg="#1E1E1E", fg="white")
task_entry.pack(pady=10)

priority_var = tk.StringVar(value="Medium")
priority_menu = tk.OptionMenu(root, priority_var, "High", "Medium", "Low")
priority_menu.pack(pady=5)

button_frame = tk.Frame(root, bg="#121212")
button_frame.pack(pady=10)

add_button = tk.Button(button_frame, text="Add Task", command=add_task, bg="#3A3A3A", fg="white")
delete_button = tk.Button(button_frame, text="Delete Task", command=delete_task, bg="#3A3A3A", fg="white")
complete_button = tk.Button(button_frame, text="Mark Complete", command=toggle_task_completion, bg="#3A3A3A", fg="white")
edit_button = tk.Button(button_frame, text="Edit Task", command=edit_task, bg="#3A3A3A", fg="white")

add_button.pack(side=tk.LEFT, padx=5)
delete_button.pack(side=tk.LEFT, padx=5)
complete_button.pack(side=tk.LEFT, padx=5)
edit_button.pack(side=tk.LEFT, padx=5)

init_db()
load_tasks()
root.mainloop()


In [15]:
import tkinter as tk
from tkinter import messagebox, simpledialog
import sqlite3
import schedule
import time
import threading
import speech_recognition as sr
import plyer
from datetime import datetime

# Database setup
def init_db():
    conn = sqlite3.connect("tasks.db")
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS tasks (
            id INTEGER PRIMARY KEY,
            task TEXT NOT NULL,
            completed BOOLEAN NOT NULL DEFAULT 0,
            priority TEXT NOT NULL DEFAULT 'Medium',
            due_date TEXT,
            category TEXT NOT NULL DEFAULT 'General',
            user TEXT NOT NULL DEFAULT 'guest'
        )
    """)
    conn.commit()
    conn.close()

def add_task():
    task = task_entry.get()
    if task:
        priority = priority_var.get()
        due_date = simpledialog.askstring("Due Date", "Enter due date (optional, YYYY-MM-DD):")
        category = simpledialog.askstring("Category", "Enter task category:")
        user = "guest"
        conn = sqlite3.connect("tasks.db")
        cursor = conn.cursor()
        cursor.execute("INSERT INTO tasks (task, priority, due_date, category, user) VALUES (?, ?, ?, ?, ?)", (task, priority, due_date, category, user))
        conn.commit()
        conn.close()
        load_tasks()
        task_entry.delete(0, tk.END)
    else:
        messagebox.showwarning("Warning", "You must enter a task!")

def delete_task():
    try:
        selected_task = task_listbox.get(task_listbox.curselection())
        task_id = selected_task.split(".")[0]
        conn = sqlite3.connect("tasks.db")
        cursor = conn.cursor()
        cursor.execute("DELETE FROM tasks WHERE id = ?", (task_id,))
        conn.commit()
        conn.close()
        load_tasks()
    except IndexError:
        messagebox.showwarning("Warning", "You must select a task to delete!")

def toggle_task_completion():
    try:
        selected_task = task_listbox.get(task_listbox.curselection())
        task_id = selected_task.split(".")[0]
        conn = sqlite3.connect("tasks.db")
        cursor = conn.cursor()
        cursor.execute("UPDATE tasks SET completed = NOT completed WHERE id = ?", (task_id,))
        conn.commit()
        conn.close()
        load_tasks()
    except IndexError:
        messagebox.showwarning("Warning", "You must select a task to mark as complete!")

def edit_task():
    try:
        selected_task = task_listbox.get(task_listbox.curselection())
        task_id = selected_task.split(".")[0]
        new_task = simpledialog.askstring("Edit Task", "Enter new task text:")
        if new_task:
            conn = sqlite3.connect("tasks.db")
            cursor = conn.cursor()
            cursor.execute("UPDATE tasks SET task = ? WHERE id = ?", (new_task, task_id))
            conn.commit()
            conn.close()
            load_tasks()
    except IndexError:
        messagebox.showwarning("Warning", "You must select a task to edit!")

def load_tasks():
    task_listbox.delete(0, tk.END)
    conn = sqlite3.connect("tasks.db")
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM tasks")
    tasks = cursor.fetchall()
    conn.close()
    
    for task in tasks:
        if len(task) < 6:
            print("Unexpected task format:", task)
            continue

        display_text = f"{task[0]}. {task[1]} [Priority: {task[3]}] [Due: {task[4] if task[4] else 'N/A'}] [Category: {task[5]}]"
        if task[2]:
            display_text += " ✅"
        task_listbox.insert(tk.END, display_text)

def check_due_tasks():
    conn = sqlite3.connect("tasks.db")
    cursor = conn.cursor()
    cursor.execute("SELECT task, due_date FROM tasks WHERE completed=0")
    tasks = cursor.fetchall()
    conn.close()
    today = datetime.today().strftime('%Y-%m-%d')
    for task, due_date in tasks:
        if due_date and due_date == today:
            plyer.notification.notify(title="Task Reminder", message=f"{task} is due today!", timeout=5)

def run_scheduler():
    while True:
        schedule.run_pending()
        time.sleep(60)

schedule.every().day.at("09:00").do(check_due_tasks)
threading.Thread(target=run_scheduler, daemon=True).start()

# GUI Setup
root = tk.Tk()
root.title("Advanced To-Do List")
root.configure(bg="#121212")

frame = tk.Frame(root, bg="#121212")
frame.pack(pady=10)

task_listbox = tk.Listbox(frame, width=60, height=10, bg="#1E1E1E", fg="white")
task_listbox.pack(side=tk.LEFT, fill=tk.BOTH)
scrollbar = tk.Scrollbar(frame)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
task_listbox.config(yscrollcommand=scrollbar.set)
scrollbar.config(command=task_listbox.yview)

task_entry = tk.Entry(root, width=50, bg="#1E1E1E", fg="white")
task_entry.pack(pady=10)

priority_var = tk.StringVar(value="Medium")
priority_menu = tk.OptionMenu(root, priority_var, "High", "Medium", "Low")
priority_menu.pack(pady=5)

button_frame = tk.Frame(root, bg="#121212")
button_frame.pack(pady=10)

add_button = tk.Button(button_frame, text="Add Task", command=add_task, bg="#3A3A3A", fg="white")
delete_button = tk.Button(button_frame, text="Delete Task", command=delete_task, bg="#3A3A3A", fg="white")
complete_button = tk.Button(button_frame, text="Mark Complete", command=toggle_task_completion, bg="#3A3A3A", fg="white")
edit_button = tk.Button(button_frame, text="Edit Task", command=edit_task, bg="#3A3A3A", fg="white")

add_button.pack(side=tk.LEFT, padx=5)
delete_button.pack(side=tk.LEFT, padx=5)
complete_button.pack(side=tk.LEFT, padx=5)
edit_button.pack(side=tk.LEFT, padx=5)

init_db()
load_tasks()
root.mainloop()


Unexpected task format: (1, 'i will spleep', 1, 'High', '2-2-2002')
Unexpected task format: (2, 'wake up', 1, 'Medium', '234545')


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Next-Level\AppData\Local\Programs\Python\Python313\Lib\tkinter\__init__.py", line 2068, in __call__
    return self.func(*args)
           ~~~~~~~~~^^^^^^^
  File "C:\Users\Next-Level\AppData\Local\Temp\ipykernel_31220\527066810.py", line 38, in add_task
    cursor.execute("INSERT INTO tasks (task, priority, due_date, category, user) VALUES (?, ?, ?, ?, ?)", (task, priority, due_date, category, user))
    ~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
sqlite3.OperationalError: table tasks has no column named category


KeyboardInterrupt: 